# Database Population

Here, we are going to take the augmented dataset that we created in the previous Jupyter notebook, and store it in a relational database. We put the database in 3rd normal form, in order to reduce redundancy, and enhance data integrity. We use the sqlite3 Python package, which allows for lightweight and in-memory relational database creation, which is the most natural form for this database to take.

In [1]:
import pandas as pd
import sqlite3

Since we are in the process of writing the database, we can create a virtual connection to an in-memory database, and then dump out the contents of the database to a file later.

In [ ]:
conn = sqlite3.connect(':memory:')

The following cells will just be some utility functions that allow us to more easily add rows into our database later.

In [3]:
def query_for_join_table(type, movie_id, val_id):
    
    # Returns the appropriately formatted query for populating the join tables between movies and other tables.
    return """
           insert into movie_has_{0} (movie_id, {0}_id) values ({1}, {2})
           """.format(type, movie_id, val_id)

In [ ]:
def query(query_string, conn=conn):
    
    # Returns the results of a particular SQL query.
    cur = conn.cursor()
    cur.execute(query_string)

    return cur.fetchall()

In [ ]:
def id_from_name(name, table):
    
    # Return the id associated with a particular name from a particular table.
    return query('select {0}_id from {0} where name="{1}"'.format(table, name.strip().title()))

In [ ]:
def add_name_to_table(name, table):
    
    # Insert the given name into the given table, and for convenience, return the id associated with the newly created name.
    query('insert into {0} (name) values ("{1}")'.format(table, name.strip().title()))

    return id_from_name(name, table)

In [ ]:
def get_id(val, table):
    
    # If the value isn't in the table, add it, and then return the ID associated with it.
    val = val.lower().title()
    
    ids = id_from_name(val, table)

    if len(ids) == 0:

        # val doesn't exist yet, so insert them, then get back their id
        ids = add_name_to_table(val, table)

    # We should be guaranteed to have one entry in ids
    assert len(ids) == 1

    return int(ids[0][0])

In order to set up the database, we declare the schema of the database in the movie.sql file. Running that script populates the database with the appropriate tables that we can then insert into as we iterate through the dataset.

In [ ]:
# Setting up the main database, loading in the table schema
with open("movie.sql") as f:
    script = f.read()

cur = conn.cursor()
cur.executescript(script)

Running this cell will allow you to see the movie.sql file contents.

In [8]:
%pycat movie.sql

# Insertion Process
1. First, read the CSV into a pandas dataframe.
2. Second, replace all of the NaNs with empty strings.
3. Since the index of each row is unique, we can use that to represent each movie_id in the database.
4. Strip all of the appropriate fields of whitespace, canonicalize them by titling them, and then format the query appropriately.
5. For each of the fields that can have multiple values per row, independently insert each of the values into the correct table and its corresponding join table.
6. Since each of the ratings (metacritic, rotten tomatoes, and IMDB) are on different scales, normalize them before we enter them into the database.

In [ ]:
# Start reading through the big CSV
df = pd.read_csv("final.csv")

for index, row in df.iterrows():

    row = {key: (str(val).replace('"', '') if str(val) != "nan" else "") for key, val in dict(row).items()}

    movie_id = index + 1

    try:
        INSERT_MOVIE_QUERY = 'insert into movie (movie_id, origin, year, title, wiki_link, plot, imdb_rating, imdb_votes, rated, rotten_tomato_rating, metacritic_rating) values ({}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {})'.format(movie_id,
                                       '"{}"'.format(row["Origin/Ethnicity"].strip().title()) if row["Origin/Ethnicity"] else "NULL",
                                       int(row["Release Year"]) if row["Release Year"] else "NULL",
                                       '"{}"'.format(row["Title"].strip().title()) if row["Title"] else "NULL",
                                       '"{}"'.format(row["Wiki Page"]) if row["Wiki Page"] else "NULL",
                                       '"{}"'.format(row["Plot"].strip().capitalize()) if row["Plot"] else "NULL",
                                       float(row["imdbRating"]) / 10 if row["imdbRating"] else "NULL",
                                       row["imdbVotes"].replace(",", "") if row["imdbVotes"] else "NULL",
                                       '"{}"'.format(row["Rated"].strip()) if row["Rated"] else "NULL",
                                       int(row["Rotten Tomatoes"]) / 100 if row["Rotten Tomatoes"] else "NULL",
                                       int(row["Metacritic"]) / 100 if row["Metacritic"] else "NULL")
    except ValueError as e:

        # There is a strange row in the dataset that just has the header names repeated again. If we run into this,
        # then move onto the next row. If that isn't the problem, then raise the exception again.
        if row["Release Year"] == "Release Year":
            continue
        else:
            raise e

    query(INSERT_MOVIE_QUERY)

    for label, table in [("Director", "director"), ("Cast", "actor"), ("Genre", "genre")]:
        
        # If there are values for this label,
        if row[label]:
            
            # Get a list of all of these values
            val_list = row[label].strip().split(',')
            for val in val_list:
                
                # If the value is a valid one,
                if val:
                    
                    # Add it to the appropriate table.
                    query(query_for_join_table(table, movie_id, get_id(val, table)))

Our database now holds all of the relevant data from the CSVs, so we can dump it out to a SQL file that we can later reload into memory.

In [ ]:
with open('dump.sql', 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' % line)